# Demonstration: Associative memory of a Hopfield network

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

### Create memory patterns

Here we use 5x5 images to store different patterns. The memories that we will train the binary Hopfield network on correspond to letters.

In [ ]:
lettersX = {}
lettersX['A'] = """
.XX..
X..X.
XXXX.
X..X.
X..X.
"""
lettersX['M'] = """
X...X
XX.XX
X.X.X
X...X
X...X
"""
lettersX['P'] = """
XXXX.
X..X.
XXXX.
X....
X....
"""
lettersX['H'] = """
.X..X
.X..X
.XXXX
.X..X
.X..X
"""
lettersX['Y'] = """
X...X
.X.X.
..X..
..X..
..X..
"""
lettersX['S'] = """
XXXXX
X....
XXXXX
....X
XXXXX
"""

In [ ]:
def toHopfieldState(patternX):
    return np.array([+1 if c=='X' else -1 for c in patternX.replace('\n','')])

In [ ]:
letters = {}
for key, value in lettersX.items():
    letters[key] =  toHopfieldState(value)

In [ ]:
letters['A']

In [ ]:
def displayHopfieldState(pattern, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(3,3))
    ax.imshow(pattern.reshape((5,5)), cmap=plt.cm.binary, interpolation='nearest')

In [ ]:
fig,axs = plt.subplots(1,5, figsize=(10,4))
for ax, key in zip(axs,['M','P','P','H','S']):
    displayHopfieldState(letters[key],ax)

### Train the network

Let us just train on four selected letters.

In [ ]:
memories = np.array([letters['M'], letters['P'], letters['H'], letters['S'], ])

In [ ]:
memories.shape

In [ ]:
def trainHopfield(patterns):
    M, C = patterns.shape
    W = np.zeros((C,C))
    # Hebbian learning
    for p in patterns:
        W += np.outer(p,p)
    W[np.diag_indices(C)] = 0
    # Scaling of weights by number of train patterns
    return W / M

In [ ]:
HopfieldWeights = trainHopfield(memories)
HopfieldWeights.shape

In [ ]:
plt.imshow(HopfieldWeights, interpolation='nearest')
plt.colorbar();

### Energy of Hopfield network states

In [ ]:
# Measure the energy of a Hopfield state
def energyHopfieldState(W, p):
    return -0.5 * np.dot(np.dot(p.T, W), p)

In [ ]:
fig,axs = plt.subplots(1,4, figsize=(10,4))
for ax, key in zip(axs,['M','P','H','S']):
    displayHopfieldState(letters[key],ax)
    ax.set_title(f'Memory (E={(energyHopfieldState(HopfieldWeights, letters[key]))})')
fig,axs = plt.subplots(1,2, figsize=(5,4))
for ax, key in zip(axs,['A','Y']):
    displayHopfieldState(letters[key],ax)
    ax.set_title(f'Non-memory (E={(energyHopfieldState(HopfieldWeights, letters[key]))})')

### Distorted letters

In [ ]:
def distort(p, size=5):
    pcopy = p.copy()
    inds = np.arange(25)
    np.random.shuffle(inds)
    for ibit in inds[:size]:
        #print(f'Flipping bit {ibit}')
        pcopy[ibit] *= -1
    return pcopy

In [ ]:
letters['A']

In [ ]:
pDistort = distort(letters['A'])

In [ ]:
pDistort

In [ ]:
np.sum(pDistort!=letters['A'])

In [ ]:
distortedLetters = {}
for key, value in letters.items():
    distortedLetters[key] =  distort(value)

In [ ]:
fig,axs = plt.subplots(1,6, figsize=(15,4))
for ax, (key, dpattern) in zip(axs,distortedLetters.items()):
    displayHopfieldState(dpattern,ax)
    ax.set_title(f'Distorted {key} (E={(energyHopfieldState(HopfieldWeights, dpattern))})')

### Recall memory

In [ ]:
def recall(W, p, steps=5):
    pcopy = p.copy()
    for _ in range(steps):
        pcopy = np.sign(np.dot(pcopy, W))
    return pcopy

In [ ]:
p_recall = recall(HopfieldWeights, distortedLetters['M'])

In [ ]:
displayHopfieldState(p_recall)

In [ ]:
Nsteps = 5
fig,axs = plt.subplots(Nsteps,6, figsize=(15,15))
for istep in range(Nsteps):
    for ax, (key, dpattern) in zip(axs[istep,:],distortedLetters.items()):
        p_recall = recall(HopfieldWeights, dpattern, steps=istep)
        displayHopfieldState(p_recall,ax)
        if istep==0:
            ax.set_title(f'Distorted {key} (E={(energyHopfieldState(HopfieldWeights, p_recall))})')
        else:
            ax.set_title(f'Step {istep} (E={(energyHopfieldState(HopfieldWeights, p_recall))})')